# Downloading videos GB1 por Mestre Carlos Gracie Jr. 

### Getting the videos ids

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def load_cookies(driver, cookies_file):
    with open(cookies_file, 'r', encoding='utf-8') as f:
        cookies = json.load(f)
    for cookie in cookies:
        if 'sameSite' in cookie:
            if cookie['sameSite'] == 'unspecified':
                cookie['sameSite'] = 'None'
            elif cookie['sameSite'] == 'no_restriction':
                cookie['sameSite'] = 'None'
            elif cookie['sameSite'] == 'lax':
                cookie['sameSite'] = 'Lax'
            elif cookie['sameSite'] == 'strict':
                cookie['sameSite'] = 'Strict'
        if 'expiry' in cookie:
            cookie['expires'] = cookie.pop('expiry')
        try:
            driver.add_cookie(cookie)
        except Exception as e:
            print(f"Não foi possível adicionar o cookie {cookie['name']}: {e}")

driver.get('https://online.graciebarra.com.br/')
time.sleep(3) 

load_cookies(driver, 'cookies.json')

driver.get('https://online.graciebarra.com.br/')
time.sleep(3)

url = 'https://online.graciebarra.com.br/programs/collection-ds849l4dl38?cid=2990058&permalink=semana-1-aula-a-tcnica-1-46859-b43855'
driver.get(url)

xpath_div = '/html/body/main/div[1]/div/turbo-frame/turbo-frame/turbo-frame/div[2]/div[2]/div[2]/turbo-frame/div/div[2]'

ids = []

try:
    element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, xpath_div))
    )

    div_element = driver.find_element(By.XPATH, xpath_div)

    items = div_element.find_elements(By.CLASS_NAME, 'contents')

    for item in items:
        inner_item_div = item.find_elements(By.TAG_NAME, 'div')[0]
        if inner_item_div.get_attribute('data-controller'):  
            id = item.get_attribute('id')
            if id:
                ids.append(id.replace('program_playlist_chapter_', ''))
except Exception as e:
    print("Ocorreu um erro:", e)
    
print('IDS: ', ids)

IDS:  ['2990058', '2990059', '2990060', '2990062', '2990063', '2990064', '2990067', '2990068', '2990069', '2990071', '2990072', '2990073', '2990076', '2990077', '2990078', '2990080', '2990081', '2990082', '2990085', '2990086', '2990087', '2990089', '2990090', '2990091', '2990094', '2990095', '2990096', '2990098', '2990099', '2990100', '2990103', '2990104', '2990105', '2990107', '2990108', '2990109', '2990112', '2990113', '2990114', '2990116', '2990117', '2990118', '2990121', '2990122', '2990123', '2990125', '2990126', '2990127', '2990130', '2990131', '2990132', '2990134', '2990135', '2990136', '2990139', '2990140', '2990141', '2990143', '2990144', '2990145', '2990148', '2990149', '2990150', '2990152', '2990153', '2990154', '2990157', '2990158', '2990159', '2990161', '2990162', '2990163', '2990166', '2990167', '2990168', '2990170', '2990171', '2990172', '2990175', '2990176', '2990177', '2990179', '2990180', '2990181', '2990184', '2990185', '2990186', '2990188', '2990189', '2990190', '29

### Downloading videos and annotate descriptions

In [9]:
from yt_dlp import YoutubeDL
import csv

try:
    with open('../dataset/gb1/annotations.csv', mode='w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['id', 'description']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for id in ids:
            url = 'https://online.graciebarra.com.br/programs/collection-ds849l4dl38?cid=' + id
            driver.get(url)
            
            description_xpath = '/html/body/main/div[1]/div/turbo-frame/turbo-frame/turbo-frame/div[2]/div[4]/div[1]/turbo-frame/ds-show-more/div'

            description_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, description_xpath))
            )
            
            description_text = description_element.text.strip()

            writer.writerow({'id': id, 'description': description_text})
            
            source_xpath = '/html/body/main/div[1]/div/turbo-frame/turbo-frame/turbo-frame/div[2]/div[2]/turbo-frame/div/video-player/video/source'
            
            source = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, source_xpath))
            )
            
            source_src = source.get_attribute('src')
            
            ydl_opts = {
                'outtmpl': f'../dataset/gb1/videos/video_{id}.mp4',  
                'format': 'best',             
                'quiet': False,                
            }
            
            with YoutubeDL(ydl_opts) as ydl:
                ydl.download([source_src])
            
            print(f'Vídeo {id} baixado com sucesso!')
except Exception as e:
    print("Ocorreu um erro:", e)
finally:
    driver.quit()